#Import Python libraries
##rdflib - https://pypi.python.org/pypi/rdflib

In [1]:
import os
import rdflib as rdf
#import csv for reading csv files
import csv

#Create new RDF graph

In [2]:
g = rdf.Graph()

#Add namespaces
## Add a namespace for each one in the object model

In [3]:
nidm = rdf.Namespace("http://nidm.nidash.org/")
prov = rdf.Namespace("http://www.w3.org/ns/prov#")
ncit = rdf.Namespace("http://ncitt.ncit.nih.gov/")
nidash = rdf.Namespace("http://purl.org/nidash/nidm/")
fbirn = rdf.Namespace("http://www.birncommunity.org/collaborators/function-birn/")
list(g.namespaces())

[('xml', rdflib.term.URIRef(u'http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef(u'http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('xsd', rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#')),
 ('rdfs', rdflib.term.URIRef(u'http://www.w3.org/2000/01/rdf-schema#'))]

#Bind namespaces to graph

In [4]:
g.bind('nidm', nidm)
g.bind('prov', prov)
g.bind('ncit', ncit)
g.bind('nidash', nidash)
g.bind('fbirn', fbirn)
list(g.namespaces())

[('xml', rdflib.term.URIRef(u'http://www.w3.org/XML/1998/namespace')),
 ('fbirn',
  rdflib.term.URIRef(u'http://www.birncommunity.org/collaborators/function-birn/')),
 ('ncit', rdflib.term.URIRef(u'http://ncitt.ncit.nih.gov/')),
 ('rdfs', rdflib.term.URIRef(u'http://www.w3.org/2000/01/rdf-schema#')),
 ('prov', rdflib.term.URIRef(u'http://www.w3.org/ns/prov#')),
 ('nidash', rdflib.term.URIRef(u'http://purl.org/nidash/nidm/')),
 ('nidm', rdflib.term.URIRef(u'http://nidm.nidash.org/')),
 ('rdf', rdflib.term.URIRef(u'http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('xsd', rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#'))]

#Create function to create nodes and links in RDF graph

In [5]:
def nidm_add_nodes(graph,line):
    #Create new node for Activity
    g.add((nidash["activity_"+line["subjectID"]], rdf.RDF.type, prov["Activity"]))
    g.add((nidash["activity_"+line["subjectID"]], prov["label"], rdf.Literal("Assessment data collection")))
    #Add new node for Agent
    g.add((nidash["agent_"+line["subjectID"]], rdf.RDF.type, prov["Agent"]))
    g.add((nidash["agent_"+line["subjectID"]], prov["type"], prov["Person"]))
    g.add((nidash["agent_"+line["subjectID"]], ncit["subjectID"], rdf.Literal(line["subjectID"])))
    g.add((nidash["agent_"+line["subjectID"]], ncit["species"], rdf.Literal("Homo sapien")))
    #Create collection for FBIRN NAART variables
    g.add((rdf.Literal("collection_fbirn:NAART_"+line["subjectID"]), rdf.RDF.type, prov["collection"]))
    g.add((rdf.Literal("collection_fbirn:NAART_"+line["subjectID"]), prov["label"], rdf.Literal("fbirn:NAART")))
    #Create an entity for the NAART variables
    g.add((nidash["entity_"+line["subjectID"]], rdf.RDF.type, prov["Entity"]))
    g.add((nidash["entity_"+line["subjectID"]], fbirn["FSIQ"], rdf.Literal(line["FSIQ"])))
    g.add((nidash["entity_"+line["subjectID"]], fbirn["PIQ"], rdf.Literal(line["PIQ"])))
    g.add((nidash["entity_"+line["subjectID"]], fbirn["VIQ"], rdf.Literal(line["VIQ"])))
    #Connect NAART variable entity to NAART collection
    g.add((rdf.Literal("collection_fbirn:NAART_"+line["subjectID"]), prov["hadMember"], rdf.Literal("entity_"+line["subjectID"])))
    #Create collection for FBIRN demographics assessment and entity variable and connection
    g.add((rdf.Literal("collection_fbirn:Demographics_"+line["subjectID"]), rdf.RDF.type, prov["collection"]))
    g.add((rdf.Literal("collection_fbirn:Demographics_"+line["subjectID"]), prov["label"], rdf.Literal("fbirn:Demographics")))
    g.add((nidash["entity_"+line["subjectID"]], rdf.RDF.type, prov["Entity"]))
    g.add((nidash["entity_"+line["subjectID"]], ncit["Age"], rdf.Literal(line["Age"])))
    g.add((nidash["entity_"+line["subjectID"]], ncit["Handedness"], rdf.Literal(line["Handedness"])))
    g.add((rdf.Literal("collection_fbirn:Demographics_"+line["subjectID"]), prov["hadMember"], rdf.Literal("entity_"+line["subjectID"])))
    #Create connections between activity, agent, and collections
    g.add((nidash["activity_"+line["subjectID"]], prov["wasAssociatedWith"], rdf.Literal("collection_fbirn:NAART_"+line["subjectID"])))
    g.add((nidash["activity_"+line["subjectID"]], prov["wasAssociatedWith"], rdf.Literal("collection_fbirn:Demographics_"+line["subjectID"])))
    g.add((nidash["agent_"+line["subjectID"]], prov["wasAttributedTo"], rdf.Literal("collection_fbirn:NAART_"+line["subjectID"])))
    g.add((nidash["agent_"+line["subjectID"]], prov["wasAttributedTo"], rdf.Literal("collection_fbirn:Demographics_"+line["subjectID"])))

#Read in CSV file and convert to RDF graph

In [6]:
input_file = csv.DictReader(open("fbirn_data.csv"))
for line in input_file:
    nidm_add_nodes(g,line)  

In [7]:
print g.serialize(format='turtle')

@prefix fbirn: <http://www.birncommunity.org/collaborators/function-birn/> .
@prefix ncit: <http://ncitt.ncit.nih.gov/> .
@prefix nidash: <http://purl.org/nidash/nidm/> .
@prefix nidm: <http://nidm.nidash.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

nidash:activity_301882920 a prov:Activity ;
    prov:label "Assessment data collection" ;
    prov:wasAssociatedWith "collection_fbirn:Demographics_301882920",
        "collection_fbirn:NAART_301882920" .

nidash:activity_303269784 a prov:Activity ;
    prov:label "Assessment data collection" ;
    prov:wasAssociatedWith "collection_fbirn:Demographics_303269784",
        "collection_fbirn:NAART_303269784" .

nidash:activity_303851049 a prov:Activity ;
    prov:label "Assessment data collection" ;
    prov:wasAssoc

#Save RDF Turtle file to disk

In [8]:
with open("FBIRN_to_NIDM_RDFLib.ttl",'w') as f:
    f.write(g.serialize(format='turtle'))

#Create RDF graph from DOT file

In [9]:
from rdflib.tools import rdf2dot
with open("FBIRN_to_NIDM_RDFLib.dot",'w') as f:
    s = rdf2dot.rdf2dot(g, f)


In [10]:
import pydot
g = pydot.graph_from_dot_file('FBIRN_to_NIDM_RDFLib.dot')
g.write_png('FBIRN_to_NIDM_RDFLib.png')

True